In [1]:
import sys
print(sys.version)

3.8.2 (default, Apr  8 2021, 23:19:18) 
[Clang 12.0.5 (clang-1205.0.22.9)]


In [1]:
from decouple import config
from qiskit import IBMQ
from datetime import datetime


In [2]:
def pdt():
    print(datetime.now())

In [4]:
IBMQ

In [5]:
IBMQ.save_account(config('IBM_API_KEY'), overwrite=True)

In [2]:
IBMQ.load_account()
IBMQ.providers()
provider = IBMQ.get_provider(hub='strangeworks-hub', group='science-team', project='science-test')
print(provider.backends())

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='strangeworks-hub', group='science-team', project='science-test')>, <IBMQBackend('ibmq_montreal') from IBMQ(hub='strangeworks-hub', group='science-team', project='science-test')>, <IBMQBackend('ibm_nairobi') from IBMQ(hub='strangeworks-hub', group='science-team', project='science-test')>]


In [12]:
import qiskit as qs
from qiskit.compiler import transpile, assemble
qr = qs.QuantumRegister(2)
cr = qs.ClassicalRegister(2)
qc = qs.QuantumCircuit(qr,cr)
qc.h(0)
qc.cx(0, 1)
qc.measure(qr,cr)

In [13]:
backend = provider.get_backend('ibmq_qasm_simulator')
mapped_circuit = transpile(qc, backend=backend)
qobj = assemble(mapped_circuit, backend=backend, shots=1024)
job = backend.run(qobj)

In [14]:
job.status()

<JobStatus.DONE: 'job has successfully run'>

In [15]:
result = job.result()
counts = result.get_counts()
print(counts)

{'00': 528, '11': 496}


In [4]:
provider.has_service('runtime')

True

In [8]:
pdt()
provider.runtime.pprint_programs()

2021-06-11 14:53:17.793110
circuit-runner:
  Name: circuit-runner
  Description: A runtime program that takes one or more circuits, compiles them, executes them, and optionally applies measurement error mitigation.
  Version: 1
  Creation date: 2021-05-07T00:17:07Z
  Max execution time: 1800
  Input parameters:
    - circuits:
      Description: A circuit or a list of circuits.
      Type: A QuantumCircuit or a list of QuantumCircuits.
      Required: True
    - shots:
      Description: Number of repetitions of each circuit, for sampling. Default: 1024.
      Type: int
      Required: False
    - initial_layout:
      Description: Initial position of virtual qubits on physical qubits.
      Type: dict or list
      Required: False
    - layout_method:
      Description: Name of layout selection pass ('trivial', 'dense', 'noise_adaptive', 'sabre')
      Type: string
      Required: False
    - routing_method:
      Description: Name of routing pass ('basic', 'lookahead', 'stochastic', 

In [13]:
program = provider.runtime.program('sample-program')
print(program)

sample-program:
  Name: sample-program
  Description: A sample runtime program.
  Version: 1
  Creation date: 2021-06-04T14:11:19Z
  Max execution time: 300
  Input parameters:
    - iterations:
      Description: Number of iterations to run. Each iteration generates and runs a random circuit.
      Type: int
      Required: True
  Interim results:
    - iteration:
      Description: Iteration number.
      Type: int
    - counts:
      Description: Histogram data of the circuit result.
      Type: dict
  Returns:
    - -:
      Description: A string that says 'All done!'.
      Type: string


In [5]:
def interim_result_callback(job_id, interim_result):
    print(f"interim result: {interim_result}")

In [6]:
backend = provider.get_backend('ibmq_montreal')
program_inputs = {
    'iterations': 3
}
options = {'backend_name': backend.name()}
job = provider.runtime.run(program_id="sample-program",
                           options=options,
                           inputs=program_inputs,
                           callback=interim_result_callback
                          )
print(f"job id: {job.job_id()}")
result = job.result()
print(result)

job id: c35lks11vsjqegaq50sg


OSError: Could not find a suitable TLS CA certificate bundle, invalid path: /Users/dylan/repos/qiskit-test/venv/lib/python3.8/site-packages/certifi/cacert.pem

In [24]:
import requests
import json
import time


In [25]:
headers = {
    'Authorization': f"Bearer {config('IBM_API_KEY')}",
    'Content-Type': 'application/json'
}

Runtime_API_URL = "https://runtime-us-east.quantum-computing.ibm.com/"



In [26]:
res = requests.get(Runtime_API_URL + 'programs' , headers = headers)

In [27]:
import pprint
pprint.pprint(res.json())

[{'cost': 1800,
  'creationDate': '2021-05-07T00:17:07Z',
  'description': 'A runtime program that takes one or more circuits, compiles '
                 'them, executes them, and optionally applies measurement '
                 'error mitigation.',
  'id': 'circuit-runner',
  'name': 'circuit-runner',
  'parameters': '{"doc": [{"name": "circuits", "description": "A circuit or a '
                'list of circuits.", "type": "A QuantumCircuit or a list of '
                'QuantumCircuits.", "required": true}, {"name": "shots", '
                '"description": "Number of repetitions of each circuit, for '
                'sampling. Default: 1024.", "type": "int", "required": false}, '
                '{"name": "initial_layout", "description": "Initial position '
                'of virtual qubits on physical qubits.", "type": "dict or '
                'list", "required": false}, {"name": "layout_method", '
                '"description": "Name of layout selection pass (\'trivial\'

In [28]:
rt_program = "runtime-simple"
program_in = {"iterations":2}

params = json.dumps({
      "programId": rt_program,
      "hub": "strangeworks-hub",
      "group": "internal",
      "project": "default",
      "backend": "ibmq_montreal",
      "params": [
        json.dumps(program_in)
      ]
})

job_id = ''

res = requests.post(Runtime_API_URL + 'jobs', data=params, headers=headers)

In [29]:
res.status_code

200

In [30]:
pprint.pprint(res.json())

{'id': 'c2nsakqvs0sjl2qsuhj0'}


In [26]:
IBMQ.providers()

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>]

In [27]:
class Test1:
    def __init__(self, val):
        self._val = val
    @property
    def val(self):
        return self._val

In [33]:
class Test2(Test1):
    def __init__(self, thing, stonk):
        self._thing = thing
        self._stonk = stonk
    
    @property
    def thing(self):
        return self._thing

    @property
    def stonk(self):
        return self._stonk

In [34]:
aa = Test2(1, 3)

In [36]:
aa.val

AttributeError: 'Test2' object has no attribute '_val'

In [18]:
runtime_simple_data = os.path.join(os.getcwd(), "qiskit_runtime/sw_runtime_simple/sw_runtime_simple.py")
runtime_simple_json = os.path.join(os.getcwd(), "qiskit_runtime/sw_runtime_simple/sw_runtime_simple.json")

In [19]:
program_id = provider.runtime.upload_program(
    data=runtime_simple_data,
    metadata=runtime_simple_json
)

IBMQNotAuthorizedError: 'You are not authorized to upload programs.'

358.5